In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
from astropy.io import ascii
from astropy.table import Table
from tqdm import tqdm
from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel
import numpy.ma as ma
from sqlalchemy import text
import pandas as pd
import matplotlib.pylab as pl
import matplotlib as mpl
import matplotlib.ticker as mticker
from scipy import stats
import pickle


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
loc = '/Users/allybaldelli/Desktop/AMNH-stuff/Doublet-Quantifiers'
sys.path.append(os.path.abspath(loc))

# Importing scripts needed
from Doublet_Quantifiers.curvefit import *
from Doublet_Quantifiers.models import *

mypath = '/Users/allybaldelli/Desktop/AMNH-stuff/quantifying_clouds_ally'
sys.path.append(os.path.abspath(mypath))

from functions.data_tools import *
from functions.visualizing_sonora_tools import *

In [3]:

# opening convolved data dictionary 
with open(mypath + "/diamondback-data/pickle_files/convolved_data/t1500_convolve_data_dict.pkl", "rb") as f:
    convolve_data_dict = pickle.load(f)

# opening pseudo-voigt parameter DataFrame
with open(mypath + "/diamondback-data/pickle_files/parameters/t1500_2parameter_df.pkl", "rb") as f:
    parameter_df = pickle.load(f)

In [4]:
long_plot(parameter_df, convolve_data_dict, x_min = 1.23, x_max = 1.27, 
                  x_increment = 0.005, const_spacing = 2, section_spacing= 1, norm_scaling = 6e10,
                  title = "Potassium Doublets", color_by_logg = True)
#plt.savefig('/Users/allybaldelli/Desktop/AMNH-stuff/quantifying_clouds_ally/plots/long_plot2_logg.png')

In [6]:
lines_TF = False

fig = plt.figure(figsize=(8, 10), constrained_layout=True)

gs = fig.add_gridspec(4, 2, height_ratios=[1, 1, 1, 0.05], width_ratios=[1, 1])

ax = [[fig.add_subplot(gs[i, 0]), fig.add_subplot(gs[i, 1])] for i in range(3)]
########################################################################################
# Plotting each data points
# A
plot_parameter_vs_logg_fsed(ax[0][0], ax[0][1], (parameter_df.A1 + parameter_df.A2)/2,
               parameter_df.logg, parameter_df.clouds, r"$avg(A)$, Max Depth", lines = lines_TF)

# FWHM
plot_parameter_vs_logg_fsed(ax[1][0], ax[1][1], (parameter_df.FWHM1 + parameter_df.FWHM2)/2,
               parameter_df.logg, parameter_df.clouds, r"$avg(\sigma)$, FWHM", lines = lines_TF)

# mixing parameter
plot_parameter_vs_logg_fsed(ax[2][0], ax[2][1], (parameter_df.nu1 + parameter_df.nu2)/2,
               parameter_df.logg, parameter_df.clouds, r"$avg(η)$, Mixing Parameter", lines = lines_TF)


########################################################################################
# labels along the bottom
ax[2][0].set_xlabel("Gravity")
ax[2][0].set_xticklabels(logg_ticks)

ax[2][1].set_xlabel(r"$f_{sed}$")
ax[2][1].set_xticklabels(fsed_ticks)

########################################################################################
# labels along y axis
# Max depth
yticks = ax[0][0].get_yticks()
ylabel = list(yticks.copy()/1e11)
ylabel[0] = "Stronger\nabsorption\n"
ylabel[-1] = "Weaker\nabsorption"
ax[0][0].set_yticks(yticks)
ax[0][0].set_yticklabels(ylabel)
ax[0][1].set_yticks(yticks)
ax[0][1].set_yticklabels([None for i in ylabel])

# FWHM
yticks = ax[1][0].get_yticks()
ylabel = list(np.around(yticks.copy()/1e-3, 1))
ylabel[-1] = "Broader"
ylabel[0] = "Narrower"
ax[1][0].set_yticks(yticks)
ax[1][0].set_yticklabels(ylabel)
ax[1][1].set_yticks(yticks)
ax[1][1].set_yticklabels([None for i in ylabel])

# Mixing parameter
yticks = ax[2][1].get_yticks()
ylabel = list(np.around(yticks.copy(), 2))
ylabel[0] = "Less\nbroadened\n"
ylabel[-1] = "Broadened\nwings"
ax[2][0].set_yticks(yticks)
ax[2][0].set_yticklabels(ylabel)
ax[2][1].set_yticks(yticks)
ax[2][1].set_yticklabels([None for i in ylabel])


########################################################################################
# Colorbar
cax1 = fig.add_subplot(gs[3, 1])
axcb = logg_colorbar(fig, cax = cax1,  orientation='horizontal')
axcb.set_label(r'$\log(g)$', fontsize=10)  # empty label

cax1 = fig.add_subplot(gs[3, 0])
axcb = fsed_colorbar(fig, cax = cax1,  orientation='horizontal')
axcb.set_label(r'$f_{sed}$', fontsize=10)   # empty label


fig.suptitle(
    'P-Voigt Parameters vs. Gravity and Clouds \n T=1500K \n Second Doublet', fontsize=16)
# plt.tight_layout()

#plt.savefig('/Users/allybaldelli/Desktop/AMNH-stuff/quantifying_clouds_ally/plots/second_doublet_parameters.png')

Text(0.5, 0.98, 'P-Voigt Parameters vs. Gravity and Clouds \n T=1500K \n Second Doublet')

In [7]:
# plotting each doublet separately
lines_TF = False

fig = plt.figure(figsize=(16, 10), constrained_layout=True)

gs = fig.add_gridspec(4, 4, height_ratios=[
                      1, 1, 1, 0.05], width_ratios=[1, 1, 1, 1])

ax = [[fig.add_subplot(gs[i, 0]), fig.add_subplot(gs[i, 1]), fig.add_subplot(
    gs[i, 2]), fig.add_subplot(gs[i, 3])] for i in range(3)]
########################################################################################
# Plotting each data points
# A
# doublet 1
plot_parameter_vs_logg_fsed(ax[0][0], ax[0][2], parameter_df.A1,
               parameter_df.logg, parameter_df.clouds, r"$A$, Max Depth", lines = lines_TF)

# doublet 2
plot_parameter_vs_logg_fsed(ax[0][1], ax[0][3], parameter_df.A2,
               parameter_df.logg, parameter_df.clouds, r"", lines = lines_TF)


# FWHM
# doublet 1
plot_parameter_vs_logg_fsed(ax[1][0], ax[1][2], parameter_df.FWHM1,
               parameter_df.logg, parameter_df.clouds, r"$\sigma$, FWHM", lines = lines_TF)

# doublet 2
plot_parameter_vs_logg_fsed(ax[1][1], ax[1][3], parameter_df.FWHM2,
               parameter_df.logg, parameter_df.clouds, r"", lines = lines_TF)


# mixing parameter
# doublet 1
plot_parameter_vs_logg_fsed(ax[2][0], ax[2][2], parameter_df.nu1,
               parameter_df.logg, parameter_df.clouds, r"$η$, Mixing Parameter", lines = lines_TF)

# doublet 2
plot_parameter_vs_logg_fsed(ax[2][1], ax[2][3], parameter_df.nu2,
               parameter_df.logg, parameter_df.clouds, r"", lines = lines_TF)


########################################################################################
# labels along the bottom
ax[2][0].set_xlabel("Gravity")
ax[2][0].set_xticklabels(logg_ticks)
ax[2][1].set_xlabel("Gravity")
ax[2][1].set_xticklabels(logg_ticks)

ax[2][2].set_xlabel(r"$f_{sed}$")
ax[2][2].set_xticklabels(fsed_ticks)
ax[2][3].set_xlabel(r"$f_{sed}$")
ax[2][3].set_xticklabels(fsed_ticks)

########################################################################################
# labels along y axis # need to fix power
# Max depth
yticks = ax[0][0].get_yticks()
ylabel = list(yticks.copy()/1e11)
ylabel[0] = "stronger\nabsorption\n"
ylabel[-1] = "weaker\nabsorption"
ax[0][0].set_yticks(yticks)
ax[0][0].set_yticklabels(ylabel)
ax[0][1].set_yticks(yticks)  # setting ticks to none for the rest of the rows
ax[0][1].set_yticklabels([None for i in ylabel])
ax[0][2].set_yticks(yticks)
ax[0][2].set_yticklabels([None for i in ylabel])
ax[0][3].set_yticks(yticks)
ax[0][3].set_yticklabels([None for i in ylabel])

# FWHM
yticks = ax[1][1].get_yticks()
ylabel = list(np.around(yticks.copy()/1e-3, 1))
ylabel[0] = "broader"
ylabel[-1] = "narrower"
ax[1][0].set_yticks(yticks)
ax[1][0].set_yticklabels(ylabel)
ax[1][1].set_yticks(yticks)  # setting ticks to none for the rest of the rows
ax[1][1].set_yticklabels([None for i in ylabel])
ax[1][2].set_yticks(yticks)
ax[1][2].set_yticklabels([None for i in ylabel])
ax[1][3].set_yticks(yticks)
ax[1][3].set_yticklabels([None for i in ylabel])

# Mixing parameter
yticks = ax[2][0].get_yticks()
ylabel = list(np.around(yticks.copy(), 2))
ylabel[0] = "less\nbroadened\n"
ylabel[-1] = "\nbroadened\nwings"
ax[2][0].set_yticks(yticks)
ax[2][0].set_yticklabels(ylabel)
ax[2][1].set_yticks(yticks)  # setting ticks to none for the rest of the rows
ax[2][1].set_yticklabels([None for i in ylabel])
ax[2][2].set_yticks(yticks)
ax[2][2].set_yticklabels([None for i in ylabel])
ax[2][3].set_yticks(yticks)
ax[2][3].set_yticklabels([None for i in ylabel])


########################################################################################
# Colorbar
cax1 = fig.add_subplot(gs[3, 2])
axcb = logg_colorbar(fig, cax1,  orientation='horizontal')
axcb.set_label(r'$\log(g)$', fontsize=10)  # empty label


cax1 = fig.add_subplot(gs[3, 0])
axcb = fsed_colorbar(fig, cax1,  orientation='horizontal')
axcb.set_label(r'$f_{sed}$', fontsize=10)   # empty label

########################################################################################
# setting titles
# set doublet title
ax[0][0].set_title("Doublet 1")
ax[0][1].set_title("Doublet 2")
ax[0][2].set_title("Doublet 1")
ax[0][3].set_title("Doublet 2")

# figure title
fig.suptitle('P-Voigt Parameters vs. Gravity and Clouds', fontsize=16)

Text(0.5, 0.98, 'P-Voigt Parameters vs. Gravity and Clouds')